# Video Classification with Pytorch

In this tutorial, I will guide video classification on [Coin Dataset](https://github.com/coin-dataset/annotations). Coin is a dataset that contains instructional videos with their corresponding action labels. There are various actions, but I choose 2 of them () for simplicity of tutorial.

Recipe Steps:
1. Download Data
2. 

In [ ]:
import os
import json
import yt_dlp
import datetime
import pandas as pd

from tqdm import tqdm

Target Id                                   0
Target Label             UseAnalyticalBalance
Action Id                                  65
Action Label    clean inner wall of container
Name: 0, dtype: object

In [ ]:
# sanity check
for i in classification_dataloader_train:
    inputs, labels = i
    inputs = inputs.to(device)
    labels = labels.to(device)
    print(inputs.shape)
    print(labels)

    preds = model(inputs)
    print(preds.shape)
    break

In [11]:
import torch
import torchvision


def read_video(video_path):
    stream = "video"
    video = torchvision.io.VideoReader(video_path, stream)
    video.get_metadata()

    video.set_current_stream("audio")

    frames = []
    for frame in video:
        frames.append(frame['data'])
    print(len(frames))
    frames = torch.stack(frames)
    print(frames.shape)

read_video("/home/emre/workspace/medium_repos/video_classification_coin_dataset/coin_subset/0/_6TyXxLWYd0.mp4")

RuntimeError: Not compiled with video_reader support, to enable video_reader support, please install ffmpeg (version 4.2 is currently supported) and build torchvision from source.

In [3]:
!conda install -y -c conda-forge ffmpeg

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/emre/anaconda3/envs/coqui

  added / updated specs:
    - ffmpeg


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       h516909a_3         398 KB  conda-forge
    ca-certificates-2022.6.15  |       ha878542_0         149 KB  conda-forge
    certifi-2022.6.15          |   py38h578d9bd_0         155 KB  conda-forge
    ffmpeg-4.2.2               |       h20bf706_0        59.6 MB
    freetype-2.10.4            |       h7ca028e_0         912 KB  conda-forge
    gmp-6.2.1                  |       h58526e2_0         806 KB  conda-forge
    gnutls-3.6.13              |       h85f3911_1         2.0 MB  conda-forge
    libo

In [10]:
!pip install torchvision==0.12.0 --force-reinstall

  Using cached torchvision-0.12.0-cp38-cp38-manylinux1_x86_64.whl (21.0 MB)
  Using cached typing_extensions-4.3.0-py3-none-any.whl (25 kB)
     |████████████████████████████████| 3.2 MB 182 kB/s eta 0:00:01
     |████████████████████████████████| 17.1 MB 46 kB/s eta 0:00:015
  Using cached torch-1.11.0-cp38-cp38-manylinux1_x86_64.whl (750.6 MB)
     |████████████████████████████████| 62 kB 71 kB/s eta 0:00:011
     |████████████████████████████████| 139 kB 53 kB/s eta 0:00:01
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Using cached charset_normalizer-2.1.0-py3-none-any.whl (39 kB)
     |████████████████████████████████| 160 kB 86 kB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.4
    Uninstalling urllib3-1.26.4:
      Successfully uninstalled urllib3-1.26.4
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.3.0
    Uninstalling typing-extensions-4.3.0:
      Successfully uninstalled typing